In [ ]:
# --- MINI PROJET DATA ANALYSIS ---
# Google Colab Notebook
# Auteur : Salif Arnauld OUATTARA
# Objectif : Extraire et filtrer des données GeoNames pour le Burkina Faso

In [1]:
# ===============================
# 1. Importation des librairies
# ===============================
import pandas as pd
import zipfile
import requests
from io import BytesIO

In [2]:
# ===============================
# 2. Télécharger et extraire les données GeoNames du Burkina Faso
# ===============================
print("Téléchargement des données GeoNames...")
url = "https://download.geonames.org/export/dump/BF.zip"
response = requests.get(url)
with zipfile.ZipFile(BytesIO(response.content)) as z:
    z.extractall("data")  # extraction dans un dossier "data"

print("Téléchargement et extraction terminés.")

Téléchargement des données GeoNames...
Téléchargement et extraction terminés.


In [3]:
# ===============================
# 3. Lecture du fichier BF.txt
# ===============================
cols = [
    "geonameid", "name", "asciiname", "alternatenames",
    "latitude", "longitude", "feature class", "feature code",
    "country code", "cc2", "admin1 code", "admin2 code",
    "admin3 code", "admin4 code", "population", "elevation",
    "dem", "timezone", "modification date"
]
df = pd.read_csv("data/BF.txt", sep="\t", names=cols, dtype=str)

In [4]:
# ===============================
# 4. Garder uniquement les colonnes utiles et renommer
# ===============================
df_clean = df[["geonameid", "name", "latitude", "longitude"]].copy()
df_clean.columns = ["ID", "location_name", "lat", "long"]

# Sauvegarde du fichier nettoyé
df_clean.to_csv("burkina_location.csv", index=False)
print("Fichier burkina_location.csv sauvegardé.")

Fichier burkina_location.csv sauvegardé.


In [ ]:
# ===============================
# 5. Filtrages demandés
# ===============================

In [5]:
# 5.1 Extraction de 'gounghin'
df_gounghin = df_clean[df_clean["location_name"].str.lower().str.contains("gounghin")]
df_gounghin.to_csv("gounghin.csv", index=False)
print(f"{len(df_gounghin)} lignes contenant 'gounghin' extraites.")

10 lignes contenant 'gounghin' extraites.


In [6]:
# 5.2 Extraction noms commençant de A à P
mask_ap = df_clean["location_name"].str[0].str.upper().between("A", "P")
df_ap = df_clean[mask_ap]

# Latitude et longitude minimales + noms correspondants
lat_min = df_ap["lat"].astype(float).min()
lon_min = df_ap["long"].astype(float).min()
lat_min_name = df_ap[df_ap["lat"].astype(float) == lat_min]["location_name"].tolist()
lon_min_name = df_ap[df_ap["long"].astype(float) == lon_min]["location_name"].tolist()

print(f"Latitude minimale : {lat_min} -> {lat_min_name}")
print(f"Longitude minimale : {lon_min} -> {lon_min_name}")

Latitude minimale : 5.21609 -> ['Komoé']
Longitude minimale : -5.65968 -> ['Banifing']


In [7]:
# 5.3 Lieux avec lat >= 11 et lon <= 0.5
df_coord = df_clean[(df_clean["lat"].astype(float) >= 11) & (df_clean["long"].astype(float) <= 0.5)]
print(f"{len(df_coord)} lieux ont lat >= 11 et lon <= 0.5.")

9466 lieux ont lat >= 11 et lon <= 0.5.


In [9]:
# ===============================
# 6. Export vers Excel
# ===============================
with pd.ExcelWriter("mini_projet.xlsx") as writer:
    df_gounghin.to_excel(writer, sheet_name="gounghin", index=False)
    df_ap.to_excel(writer, sheet_name="A_to_P", index=False)

print("Fichier mini_projet.xlsx créé avec les feuilles 'gounghin' et 'A_to_P'.")

Fichier mini_projet.xlsx créé avec les feuilles 'gounghin' et 'A_to_P'.


In [10]:
# ===============================
# 7. Aperçu des fichiers générés
# ===============================
print("\n--- Aperçu gounghin.csv ---")
display(df_gounghin.head())

print("\n--- Aperçu A_to_P ---")
display(df_ap.head())

print("\n--- Aperçu lieux coord >= 11 & <= 0.5 ---")
display(df_coord.head())


--- Aperçu gounghin.csv ---


,ID,location_name,lat,long
153,2353306,Gounghin,12.06677,-1.42134
7269,2360473,Gounghin,12.62488,-1.36398
10260,2570204,Gounghin,12.31436,-1.379
10746,10342749,Gounghin,12.06667,-0.15
10759,10629032,BICIAB // Gounghin,12.35921,-1.54273



--- Aperçu A_to_P ---


,ID,location_name,lat,long
0,2282318,Pouéné,9.72908,-2.7866
1,2285251,Léraba Occidentale,10.28333,-5.11667
2,2287216,Kéléouoro,9.80748,-4.05023
4,2298457,Issana Bouga,10.91667,-1.18333
137,2353290,Forêt Classée de Ziga,12.47106,-1.08644



--- Aperçu lieux coord >= 11 & <= 0.5 ---


,ID,location_name,lat,long
5,2353158,Zyonguen,12.36667,-0.45
6,2353159,Zyiliwèlè,12.38333,-2.73333
7,2353160,Zyanko,12.78333,-0.41667
8,2353161,Zouta,13.14908,-1.28197
9,2353162,Zourtenga,12.95741,-1.28745
